# OLS on a System of Regressions


This notebook illustrates how we estimate a system of regressions with OLS - and test (coefficients) across the regressions. 

## Loading Packages

In [1]:
using Dates, DelimitedFiles, Statistics, LinearAlgebra

include("jlFiles/printmat.jl")
include("jlFiles/printTable.jl")
include("jlFiles/NWFn.jl")

printblue(txt) = printstyled(string(txt,"\n"),color=:blue,bold=true)

printblue (generic function with 1 method)

## Loading Data

In [2]:
x    = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)
Rme  = x[:,2]                #market excess return
Rf   = x[:,5]                #interest rate


x  = readdlm("Data/FF25Ps.csv",',') #no header line: x is matrix
R  = x[:,2:end]                     #returns for 25 FF portfolios
Re = R .- Rf                        #excess returns for the 25 FF portfolios
Re = Re[:,[1;7;13;19;25]]           #use just 5 assets to make the printing easier 

(T,n) = size(Re)                 #no. obs and  no. test assets

(388, 5)

# A Function for Joint Estimation of Several Regressions (OLS)


Consider the linear regression

$
y_{it}=\beta_i^{\prime}x_{t}+\varepsilon_{it}, 
$

where $i=1,2,..,n$ indicates $n$ different dependent variables. The regressors are the *same* across the regressions. 

The next cell defines a function for this estimation. The subsequent cell tests if the intercepts (alphas) are the same across regressions.

In [3]:
"""
    OlsSureFn(Y,X,m=0)

LS of Y on X; for one n dependent variables, same regressors

# Usage
(b,res,Yhat,Covb,R2a) = OlsSureFn(Y,X,m)

# Input
- `Y::Array`:     Txn, the n dependent variables
- `X::Array`:     Txk matrix of regressors (including deterministic ones)
- `m::Int`:       scalar, bandwidth in Newey-West  

# Output
- `b::Array`:     n*kx1, regression coefficients
- `u::Array`:     Txn, residuals Y - Yhat
- `Yhat::Array`:  Txn, fitted values X*b
- `V::Array`:     matrix, covariance matrix of vec(b)
- `R2a::Number`:  n vector, R2 value

"""
function OlsSureFn(Y,X,m=0)
    (T,n) = (size(Y,1),size(Y,2))
    k     = size(X,2)
    b     = X\Y
    Yhat  = X*b
    u     = Y - Yhat   
    g     = zeros(T,n*k)
    for i = 1:n                      
      vv      = (1+(i-1)*k):(i*k)   #1:k,(1+k):2k,...
      g[:,vv] = X.*u[:,i]           #moment conditions for Y[:,i] regression
    end    
    S0    = NWFn(g,m)            #Newey-West covariance matrix
    Sxxi  = -X'X/T 
    Sxx_1 = kron(Matrix(1.0I,n,n),inv(Sxxi))
    V     = Sxx_1 * S0 * Sxx_1/T
    R2a   = 1.0 .- var(u,dims=1)./var(Y,dims=1)   
    return b,u,Yhat,V,R2a
end

OlsSureFn

In [4]:
(b,u,yhat,V,R2a) = OlsSureFn(Re,[ones(T) Rme],1)
Stdb   = sqrt.(reshape(diag(V),2,n))          #V: 1:2 are for eq 1, 3:4 for eq 2,...
tstat  = b./Stdb       

printblue("CAPM regressions")
colNames = [string("asset ",i) for i=1:n]
rowNames = ["c","Rme"]

println("coeffs")
printTable(b,colNames,rowNames)

println("t-stats")
printTable(tstat,colNames,rowNames)

CAPM regressions
coeffs
      asset 1   asset 2   asset 3   asset 4   asset 5
c      -0.504     0.153     0.305     0.279     0.336
Rme     1.341     1.169     0.994     0.943     0.849

t-stats
      asset 1   asset 2   asset 3   asset 4   asset 5
c      -1.643     1.013     2.288     1.954     2.002
Rme    22.343    30.468    26.647    21.548    16.427



In [5]:
R = [1 0 -1 0 zeros(Int,1,2*n-4)]       #Testing if the alphas are the same

printblue("Testig if α1=α2")
println("R matrix")
printmat(R,width=4)

Γ = R*V*R'
test_stat = (R*vec(b))'inv(Γ)*(R*vec(b))
println("test-statictic of α1=α2 and 10% critical value of chi-square(1)")
printmat([test_stat 2.71])

Testig if α1=α2
R matrix
   1   0  -1   0   0   0   0   0   0   0

test-statictic of α1=α2 and 10% critical value of chi-square(1)
     6.415     2.710

